In [0]:
# # Install the PyDrive wrapper & import libraries.
# # This only needs to be done once per notebook.
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # Authenticate and create the PyDrive client.
# # This only needs to be done once per notebook.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# # Download a file based on its file ID.
# # https://drive.google.com/open?id=1fAiFI4rhtFNexUyhRRECZvW6dw8E_mS2
# # A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
# file_id = '1fAiFI4rhtFNexUyhRRECZvW6dw8E_mS2'
# downloaded = drive.CreateFile({'id': file_id})
# downloaded.GetContentFile("images_Data.zip")
# # print('Downloaded content "{}"'.format(downloaded.GetContentString()))
# !unzip images_Data.zip

In [0]:
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

In [5]:
import os
print(len(os.listdir("./images")),end=" ")
!cp -a ./extra_data/. ./images/
!cp -a ./Latest_data/. ./images/
!cp -a ./Test_im/. ./images/

print(len(os.listdir("./images")))

180
508


In [7]:
import keras as ks
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
import pickle as pkl
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC

Using TensorFlow backend.


In [0]:
from image_augmentations import augmentation
from model import create_model
from preprocessing import preprocess

In [0]:
orig_labels=[]
with open("./labels.pickle","rb") as file:
    orig_labels=pkl.load(file)
    
aug_labels=[]
with open("./extra_data.pickle","rb") as file:
    aug_labels=pkl.load(file)
    
aug_labels2=[]
with open("./data_extra.pickle","rb") as file:
    aug_labels2=pkl.load(file)

aug_labels3=[]
with open("./test_labels.pickle","rb") as file:
    aug_labels3=pkl.load(file)
    
labels=orig_labels+aug_labels+aug_labels2+aug_labels3

train_lab,test_lab=train_test_split(labels,test_size=0.3,random_state=3)#1,3,0.1,0.2,0.25,0.3

!mkdir aug_images
aug_labels=augmentation(train_lab)
train_lab=train_lab+aug_labels
!cp -a ./aug_images/. ./images/
print(len(test_lab))

In [10]:
def simplify_labels(labels):
    new_labels=[]
    for image in labels:
        for num in range(image[1][0]):
            new_labels.append((image[0],(1,[image[1][1][num]],[image[1][2][num]])))
    return new_labels

labels=simplify_labels(train_lab)
test_lab=simplify_labels(test_lab)
# np.random.shuffle(labels)
len(labels),len(test_lab)

(3702, 255)

In [14]:
model=create_model((224,224,3))

batch_size=128
epochs=9
steps_per_epoch=int(total_data/(batch_size))
steps_per_epoch

28

In [38]:
model.fit_generator(preprocess(labels,"./images/",batch_size=batch_size),steps_per_epoch=steps_per_epoch,epochs=1)

Epoch 1/1
28/28 [==============================] - 119s 4s/step - loss: 0.0996 - acc: 0.9760


In [39]:
gen=preprocess(test_lab,"./images/",batch_size=len(test_lab))
test_data,test_labels=next(gen)
preds=model.predict_classes(test_data)
print(classification_report(test_labels,preds))
(test_labels==preds.reshape((-1))).sum()/preds.shape[0]

              precision    recall  f1-score   support

           0       0.78      0.81      0.80       134
           1       0.78      0.74      0.76       121

    accuracy                           0.78       255
   macro avg       0.78      0.78      0.78       255
weighted avg       0.78      0.78      0.78       255



0.7803921568627451

In [0]:
# model.save("./model_1.h5")


# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# # Create & upload a text file.
# uploaded = drive.CreateFile({'title': 'IP_model_2.h5'})
# uploaded.SetContentFile("model_1.h5")
# uploaded.Upload()
# # print('Uploaded file with ID {}'.format(uploaded.get('id')))

In [0]:
#approach 2

In [0]:
base_model=ks.applications.VGG16(include_top=False,input_shape=(224,224,3))

In [0]:
gen=preprocess(labels,"./images/",batch_size=total_data)
data,t=next(gen)
data_feat=base_model.predict(data)
data_feat=data_feat.reshape((total_data,-1))

In [0]:
cls1=LinearSVC(max_iter =2000)
cls1.fit(data_feat,t)

In [0]:
gen=preprocess(test_lab,"./images/",batch_size=len(test_lab))
data_t,test_labels=next(gen)
test_feat=base_model.predict(data_t)
test_feat=test_feat.reshape((len(test_lab),-1))

In [31]:
print(classification_report(test_labels,cls1.predict(test_feat)))

              precision    recall  f1-score   support

           0       0.77      0.76      0.76       134
           1       0.74      0.74      0.74       121

    accuracy                           0.75       255
   macro avg       0.75      0.75      0.75       255
weighted avg       0.75      0.75      0.75       255

